In [2]:
! pip install tensorflow_addons

In [5]:
# Import Trained Model 

import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np 
import pandas as pd 


In [ ]:
# Import Model 
model = tf.keras.models.load_model('../../b_model.h5')

In [ ]:
# Write text 
import streamlit as st
st.title("Chest X-Ray Multi-Label Image Classification")
st.header("This is a multi-label image classification web-app designed to identify 13 diseases in chest X-ray images.")
uploaded_files = st.file_uploader("Please upload image file(s)", type=["jpg", "png"], accept_multiple_files= True)

In [4]:
def parse_function(filename):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_image(image_string, channels=3)
    
    # Equalize the Image 
    image_equalized = tfa.image.equalize(image_decoded)
    # Perform Gaussian Filtering of the Image
    image_gaus = tfa.image.gaussian_filter2d(image_equalized)
    
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_gaus, [224, 224])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized

In [ ]:
# Define Constants for the image batcher function
BATCH_SIZE = 32 # Big enough to not crash the processor
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically to reduce GPU and CPU idle time

In [ ]:
# Function to Generate the dataset required

def create_dataset(filenames):
    """Load and parse dataset.
    Args:
        filenames: list of image paths
        labels: numpy array of shape (BATCH_SIZE, N_LABELS)
        is_training: boolean to indicate training mode
    """
    
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
        
    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
# Define Disease labels 

labels = ['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Effusion',
 'Emphysema',
 'Fibrosis',
 'Infiltration',
 'Mass',
 'Nodule',
 'Pleural_Thickening',
 'Pneumonia',
 'Pneumothorax']

In [ ]:
# Create dataset from uploaded images
image_data = create_dataset(uploaded_files)
# Predict dataset from uploaded images 
multi_label_scores = model.predict(image_data)

In [ ]:
# Define the threshold for positive hits 
threshold = st.slider('Set threshold for positive result', 0, 100, 20)
st.write('Current probability threshold is ', threshold, '%')

In [ ]:
# See Predictions greater than x% for the provisional model and save to dataframe 
predictions = (multi_label_scores > (threshold/100)).astype(int)
#columns should be the same order of y_col
results=pd.DataFrame(predictions, columns=labels)
results["Filenames"]=uploaded_files
ordered_cols=["Filenames"]+labels
results=results[ordered_cols]

In [ ]:
# Function to take in a dataframe of binary labels from CNN and assign Triage levels in a new column ['Triage']

def triage_classifier(df):

    copy_df = df.copy() # Creates a copy of the dataframe 
    copy_df['Triage'] = np.nan # Creates a new empty column for the triage assignment 
    
    for ind in copy_df.index: # loops through dataframe based on index
        # Emergent = Atelectasis, Consolidation, Edema, Effusion, Infiltration, Pneumothorax
        if (copy_df['Atelectasis'][ind] == 1)\
            or (copy_df['Consolidation'][ind] ==1)\
                or (copy_df['Edema'][ind] == 1)\
                    or (copy_df['Effusion'][ind] == 1)\
                        or (copy_df['Infiltration'][ind] == 1)\
                            or (copy_df['Pneumothorax'][ind] == 1):
                            copy_df['Triage'][ind] = 'Emergent'

        # Acute = Mass, Pneumonia. Hernia is ignored due to insufficient sample size.                    
        elif (copy_df['Mass'][ind] == 1) or (copy_df['Pneumonia'][ind] == 1):
            copy_df['Triage'][ind] = 'Acute'

        # Chronic = Cardiomegaly, Emphysema, Fibrosis, Nodule, Pleural Thickening 
        elif (copy_df['Cardiomegaly'][ind] == 1)\
            or (copy_df['Emphysema'][ind] ==1)\
                or (copy_df['Fibrosis'][ind] == 1)\
                    or (copy_df['Nodule'][ind] == 1)\
                        or (copy_df['Pleural_Thickening'][ind] == 1):
                        copy_df['Triage'][ind] = 'Chronic'
        # If no hits, then 'No Finding' 
        else: 
            copy_df['Triage'][ind] = 'No Finding'

    return copy_df

In [ ]:
# Define Color Constants for Triage Classes 

EMERGENT =  'color: red;'
ACUTE = 'color: orange;'
CHRONIC = 'color: yellow;'
NO_FINDING = 'color: green;'

In [6]:
# Define Style functions 
def emergent_color_hits(series):
    highlight = EMERGENT
    default = ''
    return [highlight if e == 1 else default for e in series]

def acute_color_hits(series):
    highlight = ACUTE
    default = ''
    return [highlight if e == 1 else default for e in series]

def chronic_color_hits(series):
    highlight = CHRONIC
    default = ''
    return [highlight if e == 1 else default for e in series]

def triage_colors(cell_value):
    emergent_highlight = EMERGENT
    acute_highlight = ACUTE
    chronic_highlight = CHRONIC
    default = NO_FINDING
    if cell_value == 'Emergent':
        return emergent_highlight
    elif cell_value == 'Acute':
        return acute_highlight
    elif cell_value =='Chronic':
        return chronic_highlight
    else:
        return default 

In [ ]:

# Assign Triage Classifications
triaged_df = triage_classifier(results)

st.dataframe(triaged_df)

# .style
# .apply(emergent_color_hits, axis = 0, subset = ['Atelectasis', 'Consolidation', 'Edema', 'Effusion', 'Infiltration', 'Pneumothorax'])
# .apply(acute_color_hits, axis = 0, subset = ['Mass', 'Pneumonia'])
# .apply(chronic_color_hits, axis = 0, subset = ['Cardiomegaly', 'Emphysema', 'Fibrosis', 'Nodule', 'Pleural_Thickening'])
# .apply(triage_colors, axis = 1, subset = ['Triage']))